# Dog Adoption: Create and build data model and deploy into database with Python and PostgreSQL

In [1]:
# Import library
!pip3 install psycopg2-binary
import psycopg2
import pandas as pd

In [2]:
# Create function to create database

def create_database():
    # Connect to database
    conn = psycopg2.connect("host=127.0.0.1 dbname=postgres user=postgres password=********")
    # Use connection to get cursor to execute queries
    cur = conn.cursor()
    # Set automatic commit
    conn.set_session(autocommit=True)
    
    # Create database
    cur.execute("DROP DATABASE IF EXISTS dog_adoption")
    cur.execute("CREATE DATABASE dog_adoption")
    
    # Close connection to default database
    conn.close()
    
    # Connect to newly created database
    conn = psycopg2.connect("host=127.0.0.1 dbname=dog_adoption user=postgres password=bobo5956")
    # Use connection to get cursor to execute queries
    cur = conn.cursor()
    # Set automatic commit
    conn.set_session(autocommit=True)
    
    return cur, conn

In [3]:
# Create function to drop table

def drop_tables(cur, conn):
    for query in drop_table_queries:
        cur.execute(query)
        conn.commit

In [4]:
# Create function to create table

def create_table(cur, conn):
    for query in create_table_queries:
        cur.execute(query)
        conn.commit

### Read CSV files

In [5]:
# description.csv - File containing the description and details of the dog breed and more
description = pd.read_csv("/Users/katiehuang/Documents/Data Science/Projects/Data Engineering/Dog Adoption/Datasets/description.csv", low_memory=False)
description.head()


,index,id,org_id,url,breed_primary,breed_secondary,color_primary,color_secondary,color_tertiary,age,...,size,coat,name,status,posted,contact_city,contact_state,contact_zip,contact_country,stateQ
0,0,46042150,NV163,https://www.petfinder.com/dog/harley-46042150/...,American Staffordshire Terrier,Mixed Breed,White / Cream,Yellow / Tan / Blond / Fawn,NaN,Senior,...,Medium,Short,HARLEY,adoptable,2019-09-20T16:37:59+0000,Las Vegas,NV,89147.0,US,89009
1,1,46042002,NV163,https://www.petfinder.com/dog/biggie-46042002/...,Pit Bull Terrier,Mixed Breed,Brown / Chocolate,White / Cream,NaN,Adult,...,Large,Short,BIGGIE,adoptable,2019-09-20T16:24:57+0000,Las Vegas,NV,89147.0,US,89009
2,2,46040898,NV99,https://www.petfinder.com/dog/ziggy-46040898/n...,Shepherd,NaN,Brindle,NaN,NaN,Adult,...,Large,Short,Ziggy,adoptable,2019-09-20T14:10:11+0000,Mesquite,NV,89027.0,US,89009
3,3,46039877,NV202,https://www.petfinder.com/dog/gypsy-46039877/n...,German Shepherd Dog,NaN,NaN,NaN,NaN,Baby,...,Large,NaN,Gypsy,adoptable,2019-09-20T10:08:22+0000,Pahrump,NV,89048.0,US,89009
4,4,46039306,NV184,https://www.petfinder.com/dog/theo-46039306/nv...,Dachshund,NaN,NaN,NaN,NaN,Young,...,Small,Long,Theo,adoptable,2019-09-20T06:48:30+0000,Henderson,NV,89052.0,US,89009


In [6]:
description.columns

Index(['index', 'id', 'org_id', 'url', 'breed_primary', 'breed_secondary',
       'color_primary', 'color_secondary', 'color_tertiary', 'age', 'sex',
       'size', 'coat', 'name', 'status', 'posted', 'contact_city',
       'contact_state', 'contact_zip', 'contact_country', 'stateQ'],
      dtype='object')

In [7]:
# travel.csv - File containing the description and details of the dog breed and more
travel = pd.read_csv("/Users/katiehuang/Documents/Data Science/Projects/Data Engineering/Dog Adoption/Datasets/travel.csv")
travel.head()

,index,id,contact_city,contact_state,found,manual,remove,still_there
0,0,44520267,Anoka,MN,Arkansas,NaN,NaN,NaN
1,1,44698509,Groveland,FL,Abacos,Bahamas,NaN,NaN
2,2,45983838,Adamstown,MD,Adam,Maryland,NaN,NaN
3,3,44475904,Saint Cloud,MN,Adaptil,NaN,True,NaN
4,4,43877389,Pueblo,CO,Afghanistan,NaN,NaN,NaN


In [8]:
travel.columns

Index(['index', 'id', 'contact_city', 'contact_state', 'found', 'manual',
       'remove', 'still_there'],
      dtype='object')

In [9]:
# location.csv - File containing the dogs' current location
location = pd.read_csv("/Users/katiehuang/Documents/Data Science/Projects/Data Engineering/Dog Adoption/Datasets/location.csv")
location.head()

,index,location,exported,imported,total,inUS
0,0,Texas,635.0,NaN,566.0,True
1,1,Alabama,268.0,2.0,1428.0,True
2,2,North Carolina,158.0,14.0,2627.0,True
3,3,South Carolina,139.0,12.0,1618.0,True
4,4,Georgia,137.0,19.0,3479.0,True


In [10]:
location.columns

Index(['index', 'location', 'exported', 'imported', 'total', 'inUS'], dtype='object')

In [11]:
cur, conn = create_database()

If run into the following error while creating database:

`ObjectInUse: database "dog_adoption" is being accessed by other users
DETAIL:  There is 1 other session using the database.`

Then run this in psql terminal:

`REVOKE CONNECT ON DATABASE dog_adoption FROM public;`

`SELECT pg_terminate_backend(pg_stat_activity.pid) FROM pg_stat_activity WHERE pg_stat_activity.datname = 'dog_adoption';`

In [12]:
description.dtypes

index                int64
id                   int64
org_id              object
url                 object
breed_primary       object
breed_secondary     object
color_primary       object
color_secondary     object
color_tertiary      object
age                 object
sex                 object
size                object
coat                object
name                object
status              object
posted              object
contact_city        object
contact_state       object
contact_zip        float64
contact_country     object
stateQ              object
dtype: object

### Create tables

This process creates empty table only. 

In [13]:
create_description_table = ("""CREATE TABLE IF NOT EXISTS description \
                (index INT PRIMARY KEY, id INT, org_id VARCHAR, url VARCHAR, breed_primary VARCHAR, \
                breed_secondary VARCHAR, color_primary VARCHAR, color_secondary VARCHAR, color_tertiary VARCHAR, \
                age VARCHAR, sex VARCHAR, size VARCHAR, coat VARCHAR, name VARCHAR, status VARCHAR, \
                posted VARCHAR, contact_city VARCHAR, contact_state VARCHAR, contact_zip VARCHAR, \
                contact_country VARCHAR, stateQ VARCHAR)""")

In [14]:
cur.execute(create_description_table)
conn.commit()

In [15]:
travel.dtypes

index             int64
id                int64
contact_city     object
contact_state    object
found            object
manual           object
remove           object
still_there      object
dtype: object

In [16]:
create_travel_table = ("""CREATE TABLE IF NOT EXISTS travel \
                (index INT PRIMARY KEY, id INT, contact_city VARCHAR, contact_state VARCHAR, \
                found VARCHAR, manual VARCHAR, remove VARCHAR, still_there VARCHAR)""")

cur.execute(create_travel_table)
conn.commit()

In [17]:
location.dtypes

index         int64
location     object
exported    float64
imported    float64
total       float64
inUS           bool
dtype: object

In [18]:
location.head()

,index,location,exported,imported,total,inUS
0,0,Texas,635.0,NaN,566.0,True
1,1,Alabama,268.0,2.0,1428.0,True
2,2,North Carolina,158.0,14.0,2627.0,True
3,3,South Carolina,139.0,12.0,1618.0,True
4,4,Georgia,137.0,19.0,3479.0,True


In [19]:
create_location_table = ("""CREATE TABLE IF NOT EXISTS location \
                (index INT PRIMARY KEY, location VARCHAR, exported FLOAT, imported FLOAT, \
                total FLOAT, inUS BOOL)""")

cur.execute(create_location_table)
conn.commit()

### Insert rows into table

In [20]:
insert_description_table = ("""INSERT INTO description \
                (index, id, org_id, url, breed_primary, breed_secondary, \
                color_primary, color_secondary, color_tertiary, \
                age, sex, size, coat, name, status, \
                posted, contact_city, contact_state, contact_zip, \
                contact_country, stateQ)
                VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, \
                 %s, %s, %s, %s, %s, %s, %s, %s, %s, %s)""")


# Iterating through index (i) and rows in description table
for i, row in description.iterrows():
    # Pass INSERT INTO query in cur.execute() and list of data
    cur.execute(insert_description_table, list(row))

In [22]:
insert_travel_table = ("""INSERT INTO travel \
                (index, id, contact_city, contact_state, \
                found, manual, remove, still_there)
                VALUES (%s, %s, %s, %s, %s, %s, %s, %s)""")

for i, row in travel.iterrows(): # Iterating row by row in description table
    cur.execute(insert_travel_table, list(row))

In [23]:
insert_location_table = ("""INSERT INTO location \
                (index, location, exported, imported, \
                total, inUS)
                VALUES (%s, %s, %s, %s, %s, %s)""")

for i, row in location.iterrows(): # Iterating row by row in description table
    cur.execute(insert_location_table, list(row))